In [1]:
import copy
import re
from collections import defaultdict

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
df = pd.read_csv("../topic_words.csv.zip")
df.head()

,meta,topic,weight,type,word
0,env,Климат,5,0,ch4
1,env,Климат,5,0,co2
2,env,Климат,5,0,ghg protocol
3,env,Климат,5,0,scope 1
4,env,Климат,5,0,scope 2


In [3]:
df["topic"].value_counts()

Лидерство                          274
Энергия                            182
Риски                              177
Климат                             175
Социальные инвестиции              171
Обучение и развитие                160
Качество и безопасность продукт    158
Безопасность и охрана труда        145
Коренные народы                    126
Биоразнообразие                    118
Благотворительность                117
Вода                               114
Маркетинг, продажи, реклама        107
Оплата труда                       107
Отходы                             100
Воздух                              93
Отчетность и прозрачность           93
Ценовая политика                    86
Закупки и антикоррупция             85
Сервис и коммуникация               83
Рекультивация земель                77
Удовлетворенность потребителей      75
Профсоюз и Коллективный договор     73
Волонтерство                        71
Экологичность продукта              66
Доступность              

In [4]:
re_dict = {}
sheets_dict = {}
base_dict = {}
importance_dict = {}

for idx, row in df.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [5]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile(
        "((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))"
    )

In [6]:
# text = "использовать передовой зарубежный достижение первостроитель магнитка скоро сам стать творец новый уникальный опыт ведь история такой грандиозный промышленный объект создаваться столь сжатый срок впервые мировой практика разработать метод добыча железный руда открытый карьер это смелый технический новшество год позволить достигнуть магнитогорский рудник выработка млн так руда составлять весь железный руда ссср"
text = "тыл фронт трудно переоценить значимость магнитогорский металлургический комбинат год великий отечественный война хотя изначально ммк строиться производство мирный марка стать специальный стать применять военный промышленность ммк производиться магнитка стать настоящий бронев щит родина июль год т. е. месяц начало война третий мартеновский печь ммк получить первый броневой сталь"

print(text)
for key, regex in re_dict.items():
    words = regex.findall(text)
    paragraph_dict = copy.deepcopy(base_dict)
    for word in words:
        paragraph_dict[key][word] = sheets_dict[key][word]

    dist = cosine_similarity(
        X=[list(sheets_dict[key].values())], Y=[list(paragraph_dict[key].values())]
    )[0][0]
    print(f"{key} {words} {dist:.4f}")  # {len(words)/len(sheets_dict[key])*100:.2f}

тыл фронт трудно переоценить значимость магнитогорский металлургический комбинат год великий отечественный война хотя изначально ммк строиться производство мирный марка стать специальный стать применять военный промышленность ммк производиться магнитка стать настоящий бронев щит родина июль год т. е. месяц начало война третий мартеновский печь ммк получить первый броневой сталь
Климат [] 0.0000
Энергия [] 0.0000
Воздух ['промышленность'] 0.0459
Вода [] 0.0000
Отходы [] 0.0000
Биоразнообразие [] 0.0000
Рекультивация земель [] 0.0000
Экологичность продукта [] 0.0000
Здоровье и благополучие сотрудн [] 0.0000
Доступность [] 0.0000
Поставщики в целом [] 0.0000
Наем и увольнение [] 0.0000
Корпоративная культура [] 0.0000
Вовлеченность и мотивация [] 0.0000
Безопасность и охрана труда [] 0.0000
Сервис и коммуникация [] 0.0000
Персональные данные [] 0.0000
Здоровье и благополучие [] 0.0000
Маркетинг, продажи, реклама [] 0.0000
Удовлетворенность потребителей [] 0.0000
Качество и безопасность пр

In [7]:
ROWS = 100_000
df = pd.read_csv("paragraphs.csv.zip")  # , usecols=["cleaned_text"]
df = df.dropna().reset_index(drop=True)
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

/tmp/ipykernel_111652/3999245721.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragraphs.csv.zip")  # , usecols=["cleaned_text"]


(2309485, 8)

In [8]:
topics = []

i = 0
for text in tqdm(df["cleaned_text"].tolist()):
    paragraph_dict = copy.deepcopy(base_dict)
    paragraph_importance = copy.deepcopy(importance_dict)
    found_words = False
    for key, regex in re_dict.items():
        words = regex.findall(text)
        for word in words:
            weight = sheets_dict[key][word]
            paragraph_dict[key][word] = weight
            found_words = True
            paragraph_importance[key] += weight > 1

    res = {}

    i += 1

    # if i > 40 and i < 55:
    #     print()
    #     print(text)

    topic_found = False
    for key, vector in sheets_dict.items():
        if paragraph_importance[key] < 5 or not found_words:
            res[key] = 0
            continue
        sim = cosine_similarity(
            X=[list(vector.values())], Y=[list(paragraph_dict[key].values())]
        )[0][0]
        res[key] = sim
        topic_found = True

    if not found_words or not topic_found:
        res = {key: 0 for key in sheets_dict}
        topics.append({**res, **{"max_score": None, "max_topic": None}})
        continue
    #     if i > 40 and i < 55:
    #         print(key, sim, "words:", re_dict[key].findall(text))
    # if i > 40 and i < 55:
    #     print(
    #         "\nmax_score",
    #         max(res.values()), "\n"
    #         "max_topic",
    #         max(res, key=lambda k: res[k]), "\n", "-" * 50
    #     )

    topics.append(
        {
            **res,
            **{
                "max_score": max(res.values()),
                "max_topic": max(res, key=lambda k: res[k]),
            },
        }
    )

100%|██████████| 2309485/2309485 [8:11:44<00:00, 78.28it/s]   


In [9]:
topics[:10], topics[-10:]

([{'Климат': 0,
   'Энергия': 0,
   'Воздух': 0,
   'Вода': 0,
   'Отходы': 0,
   'Биоразнообразие': 0,
   'Рекультивация земель': 0,
   'Экологичность продукта': 0,
   'Здоровье и благополучие сотрудн': 0,
   'Доступность': 0,
   'Поставщики в целом': 0,
   'Наем и увольнение': 0,
   'Корпоративная культура': 0,
   'Вовлеченность и мотивация': 0,
   'Безопасность и охрана труда': 0,
   'Сервис и коммуникация': 0,
   'Персональные данные': 0,
   'Здоровье и благополучие': 0,
   'Маркетинг, продажи, реклама': 0,
   'Удовлетворенность потребителей': 0,
   'Качество и безопасность продукт': 0,
   'Ценовая политика': 0,
   'Профсоюз и Коллективный договор': 0,
   'Обучение и развитие': 0,
   'Оплата труда': 0,
   'Малый и локальный бизнес': 0,
   'Работники': 0,
   'Экология': 0,
   'Закупки и антикоррупция': 0,
   'Коренные народы': 0,
   'Волонтерство': 0,
   'Социальные инвестиции': 0,
   'Благотворительность': 0,
   'Отчетность и прозрачность': 0,
   'Права человека': 0,
   'Лидерство'

In [10]:
df["max_topic"].value_counts().sum()

KeyError: 'max_topic'

In [11]:
import pickle

with open("topics.pkl", "wb") as fp:  # Pickling
    pickle.dump(topics, fp)

In [12]:
topics = pd.DataFrame(topics)
topics.to_csv("all_topics.csv.zip")

In [16]:
df = df.join(topics)
df.to_csv("paragrapghs_with_topisc.csv.zip")

In [19]:
df.groupby("max_topic")["max_score"].nlargest(20)

max_topic                           
Безопасность и охрана труда  1906523    0.768706
                             169911     0.714485
                             1123988    0.709575
                             1816865    0.707107
                             1841954    0.707107
                                          ...   
Этика                        1696388    0.563093
                             384143     0.559471
                             858357     0.559471
                             1645436    0.559471
                             1675268    0.559471
Name: max_score, Length: 770, dtype: float64

In [29]:
df.loc[df.groupby("max_topic")["max_score"].nlargest(30).index.get_level_values(1)][
    [
        "company",
        "year",
        "sector",
        "report_type",
        "paragraph",
        "original_text",
        "cleaned_text",
        "max_score",
        "max_topic",
    ]
].to_csv("paragraph_max_values.csv.zip")

In [ ]:
df.to_csv("paragrapghs_with_topisc.csv")

In [44]:
grouped = pd.DataFrame(
    df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(" ".join)
)
grouped.head()

cleaned_text
company     year max_topic                                                                     
Alcoa, Inc. 2004 Антикоррупция                москва белый калитва восток московский офис са...
                 Безопасность и охрана труда          доход европа процент общий доход компания
                 Безопасность продукта        я очень приятно представить ваш внимание первы...
                 Благотворительность          лос летучий органический соединение в тысяча м...
                 Налоги                       возрастание выброс происходить связь применени...

In [45]:
from collections import defaultdict

topics_df = pd.concat([e_topics, s_topics, g_topics])
len_topics = defaultdict(dict)

for idx, row in tqdm(topics_df.iterrows(), total=topics_df.shape[0]):
    if not len_topics[row["topic"]].get(row["type"], False):
        len_topics[row["topic"]][row["type"]] = 0
    len_topics[row["topic"]][row["type"]] += 1

100%|██████████| 4414/4414 [00:00<00:00, 19458.61it/s]


In [46]:
len_topics

defaultdict(dict,
            {'Экология': {1: 51, 2: 8},
             'Климат': {1: 95, 2: 24},
             'Энергия': {1: 52, 2: 11},
             'Воздух': {1: 42},
             'Вода': {1: 61, 2: 8},
             'Отходы': {1: 41, 2: 10},
             'Биоразнообразие': {1: 47},
             'Социально-культурное воздействи': {1: 3},
             'Антикоррупция': {1: 50},
             'Дивиденды и акционеры': {1: 306},
             'Инновации': {1: 104},
             'Лидерство': {1: 371},
             'Отчетность и прозрачность': {1: 530},
             'Предотвращение коррупции': {1: 92},
             'Управление рисками': {1: 214},
             'Эффективность и производительность': {1: 319},
             'Безопасность и охрана труда': {1: 349},
             'Безопасность продукта': {1: 50},
             'Благотворительность': {1: 259},
             'Инвестиции и капитальные вложения': {1: 370},
             'Налоги': {1: 81},
             'Обучение и развитие': {1: 50},
        

In [47]:
from __future__ import annotations


def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent = (
        count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    )
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [55]:
scores = []

for (company, year, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

100%|██████████| 8788/8788 [01:05<00:00, 135.19it/s]


In [57]:
grouped["score"] = scores
grouped.head(20)

cleaned_text  \
company     year       max_topic                                                                              
Alcoa, Inc. 2004       Антикоррупция                      москва белый калитва восток московский офис са...   
                       Безопасность и охрана труда                доход европа процент общий доход компания   
                       Безопасность продукта              я очень приятно представить ваш внимание первы...   
                       Благотворительность                лос летучий органический соединение в тысяча м...   
                       Налоги                             возрастание выброс происходить связь применени...   
                       Отношения с потребителями                 белый калитва любучан восток москва самара   
                       Отношения с работниками            вступление выброс загрязнять вещество атмосфер...   
                       Отходы                             показатель здоровье работник безопасность труд...   
                       Предотвращение коррупции           наш преимущество заключаться производить очень...   
                       Профсоюзы и коллективные договоры  показатель экологический безопасность для комп...   
                       Управление рисками                 отчёт устойчивый развитие год отчёт ход выполн...   
BIOCAD      2019, 2018 Антикоррупция                      соответствие отчёт руководство прямой речь пре...   
                       Безопасность и охрана труда        экопроект павел яко вл быть в ранний разработк...   
                       Безопасность продукта              содержание устойчивый развитие стратегия приор...   
                       Благотворительность                август россия мир уровень вовлечённость сотруд...   
                       Вода                               характер собственность форма существенный изме...   
                       Дивиденды и акционеры              сокращать отход производство упаковка снижать ...   
                       Инвестиции и капитальные вложения  самый продавать препарат одержать победа суд п...   
                       Инновации                          генферон® ацеллбия® альгерон® эфлейра® млн руб...   
                       Климат                             о компания отход удаться ускорить синтезирован...   

                                                          score  
company     year       max_topic                                 
Alcoa, Inc. 2004       Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Налоги                                 5  
                       Отношения с потребителями              5  
                       Отношения с работниками                5  
                       Отходы                                 7  
                       Предотвращение коррупции               5  
                       Профсоюзы и коллективные договоры      5  
                       Управление рисками                     5  
BIOCAD      2019, 2018 Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Вода                                   7  
                       Дивиденды и акционеры                  5  
                       Инвестиции и капитальные вложения      5  
                       Инновации                              5  
                       Климат                                 7

In [58]:
grouped.to_csv("grouped_paragraphs.csv.zip")